In [1]:
"""
Purpose: Run the Soma Finding
Algorithm for all the cells
in our final match



"""


In [2]:
current_version = 30

In [3]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-04-14 18:02:40,545 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-14 18:02:40,547 - settings - Setting database.user to celiib
INFO - 2021-04-14 18:02:40,548 - settings - Setting database.password to newceliipass
INFO - 2021-04-14 18:02:40,552 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-14 18:02:40,553 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-14 18:02:40,566 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-14 18:02:40,953 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-14 18:02:40,956 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-14 18:02:40,957 - settings - Setting database.user to celiib
INFO - 2021-04-14 18:02:40,958 - settings - Setting database.password to newceliipass
INFO - 2021-04-14 18:02:40,959 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-14 18:02:40,959 - settings - Setting database.user to celiib
INFO - 2021-04-14 18:02:40,960 - settings - Setting database.password to newceliipass
INFO - 2021-04-14 18:02:40,963 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-14 18:02:41,385 - settings - Setting enable_python_native_blobs to True


In [4]:
test_mode = True

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-14 18:02:41,469 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-14 18:02:41,471 - settings - Setting database.user to celiib
INFO - 2021-04-14 18:02:41,472 - settings - Setting database.password to newceliipass
INFO - 2021-04-14 18:02:41,473 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-14 18:02:41,474 - settings - Setting database.user to celiib
INFO - 2021-04-14 18:02:41,475 - settings - Setting database.password to newceliipass
INFO - 2021-04-14 18:02:41,478 - settings - Setting enable_python_native_blobs to True


Sleeping 108 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-04-14 18:02:41,943 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [6]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [7]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [8]:
import meshlab
meshlab.set_meshlab_port(current_port=None)
temporary_folder = 'decimation_temp'
meshlab_scripts = {}

No port chosen so picked random port 949


In [9]:
#so that it will have the adapter defined
from datajoint_utils import *

In [10]:
@schema
class NeuronGliaNuclei(dj.Manual):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    n_glia_faces              : int unsigned                 # The number of faces that were saved off as belonging to glia
    glia_faces=NULL           : <faces>                      # faces indices that were saved off as belonging to glia (external storage)
    n_nuclei_faces            : int unsigned                 # The number of faces that were saved off as belonging to nuclie
    nuclei_faces=NULL         : <faces>                      # faces indices that were saved off as belonging to nuclei (external storage)
    """

In [11]:
# schema.external['faces'].delete(delete_external_files=True)
# schema.external['somas'].delete(delete_external_files=True)

In [12]:
# minnie.BaylorSegmentCentroid.delete()
# minnie.NeuronGliaNuclei().delete()

In [13]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30

key_source = (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") )  & (minnie.AllenProofreading() & dict(month=4,day=14,year=2021)).proj())
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691135337768422,0,0.25
864691135373423816,0,0.25
864691135373605832,0,0.25
864691135385570133,0,0.25
864691135493989520,0,0.25
864691135569218694,0,0.25
864691135639122747,0,0.25
864691135657789570,0,0.25
864691135748641321,0,0.25
864691135758712910,0,0.25


In [14]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30


import trimesh_utils as tu
import soma_extraction_utils as sm
@schema
class BaylorSegmentCentroid(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    centroid_x=NULL           : int unsigned                 # (EM voxels)
    centroid_y=NULL           : int unsigned                 # (EM voxels)
    centroid_z=NULL           : int unsigned                 # (EM voxels)
    n_vertices=NULL           : bigint                 #number of vertices
    n_faces=NULL            : bigint                  #number of faces
    mesh: <somas>  #datajoint adapter to get the somas mesh objects
    multiplicity=NULL         : tinyint unsigned             # the number of somas found for this base segment
    sdf=NULL                  : double                       # sdf width value for the soma
    volume=NULL               : double                       # the volume in billions (10*9 nm^3) of the convex hull
    max_side_ratio=NULL       : double                       # the maximum of the side length ratios used for check if soma
    bbox_volume_ratio=NULL    : double                       # ratio of bbox (axis aligned) volume to mesh volume to use for check if soma
    max_hole_length=NULL      : double                    #euclidean distance of the maximum hole size
    run_time=NULL : double                   # the amount of time to run (seconds)

    """

    key_source = (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") )  & (minnie.AllenProofreading() & dict(month=4,day=14,year=2021)).proj())
                                                                 
     

    def make(self,key):
        """
        Pseudocode: 
        1) Compute all of the
        2) Save the mesh as an h5 py file
        3) Store the saved path as the decomposition part of the dictionary and erase the vertices and faces
        4) Insert
        
        
        """
        
        #get the mesh data
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        print(key)
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces

        segment_id = key["segment_id"]

        (total_soma_list, 
         run_time, 
         total_soma_list_sdf,
         glia_pieces,
         nuclei_pieces) = sm.extract_soma_center(
                            segment_id,
                            current_mesh_verts,
                            current_mesh_faces,
            return_glia_nuclei_pieces=True,
        )
        
        # -------- 1/9 Addition: Going to save off the glia and nuclei pieces ----------- #
        """
        Psuedocode:
        For both glia and nuclie pieces
        1) If the length of array is greater than 0 --> combine the mesh and map the indices to original mesh
        2) If not then just put None     
        """
        orig_mesh = trimesh.Trimesh(vertices=current_mesh_verts,
                                   faces=current_mesh_faces)
        
        if len(glia_pieces)>0:
            glia_faces = tu.original_mesh_faces_map(orig_mesh,tu.combine_meshes(glia_pieces))
            n_glia_faces = len(glia_faces)
        else:
            glia_faces = None
            n_glia_faces = 0
            
        if len(nuclei_pieces)>0:
            nuclei_faces = tu.original_mesh_faces_map(orig_mesh,tu.combine_meshes(nuclei_pieces))
            n_nuclei_faces = len(nuclei_faces)
        else:
            nuclei_faces = None
            n_nuclei_faces = 0
            
        # --------- saving the nuclei and glia saves
        glia_path,nuclei_path = du.save_glia_nuclei_files(glia_faces=glia_faces,
                                 nuclei_faces=nuclei_faces,
                                 segment_id=segment_id)
        
        print(f" glia_path = {glia_path} \n nuclei_path = {nuclei_path}")
            
        glia_nuclei_key = dict(key,
                               ver=current_version,
                               n_glia_faces=n_glia_faces,
                               #glia_faces = glia_faces,
                               glia_faces = glia_path,
                               n_nuclei_faces = n_nuclei_faces,
                               #nuclei_faces = nuclei_faces
                               nuclei_faces = nuclei_path,
                              )
        
        NeuronGliaNuclei.insert1(glia_nuclei_key,replace=True)
        print(f"Finished saving off glia and nuclei information : {glia_nuclei_key}")
        
        # ---------------- End of 1/9 Addition --------------------------------- #
        
        
        
        print(f"Run time was {run_time} \n    total_soma_list = {total_soma_list}"
             f"\n    with sdf values = {total_soma_list_sdf}")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            

            returned_file_path = tu.write_h5_file(
                                                vertices=np.array([]),
                                                  faces=np.array([]),
                                                  segment_id=segment_id,
                                                  filename = f'{segment_id}_0.h5',
                                                    filepath=str(du.get_somas_path())
                                                 )

            
            
            insert_dict = dict(key,
                              soma_index=0,
                               ver=current_version,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               #distance_from_prediction=None,
                               #prediction_matching_index = None,
                               n_vertices=0,
                               n_faces=0,
                               mesh=returned_file_path,
                               multiplicity=0,
                               sdf = None,
                               volume = None,
                               max_side_ratio = None,
                               bbox_volume_ratio = None,
                               max_hole_length=None,
                               run_time=run_time
                              )
            
            #raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found, get the volume and side length checks
        max_side_ratio =  [np.max(sm.side_length_ratios(m)) for m in total_soma_list]
        bbox_volume_ratio =  [sm.soma_volume_ratio(m) for m in total_soma_list]
        dicts_to_insert = []


        for i,(current_soma,soma_sdf,sz_ratio,vol_ratio) in enumerate(zip(total_soma_list,total_soma_list_sdf,max_side_ratio,bbox_volume_ratio)):
            print("Trying to write off file")
            """ Currently don't need to export the meshes
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            """
            auto_prediction_center = np.mean(current_soma.vertices,axis=0) / np.array([4,4,40])
            auto_prediction_center = auto_prediction_center.astype("int")
            print(f"Predicted Coordinates are {auto_prediction_center}")
            max_hole_length = tu.largest_hole_length(current_soma)
            
            returned_file_path = tu.write_h5_file(
                                            vertices=current_soma.vertices,
                                              faces=current_soma.faces,
                                              segment_id=segment_id,
                                              filename = f'{segment_id}_{i}.h5',
                                                filepath=str(du.get_somas_path())
                                             )



            insert_dict = dict(key,
                              soma_index=i+1,
                               ver=current_version,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               n_vertices = len(current_soma.vertices),
                               n_faces = len(current_soma.faces),
                               mesh=returned_file_path,
                               multiplicity=len(total_soma_list),
                               sdf = np.round(soma_sdf,3),
                               volume = current_soma.convex_hull.volume/1000000000,
                               max_side_ratio = np.round(sz_ratio,3),
                               bbox_volume_ratio = np.round(vol_ratio,3),
                               max_hole_length = np.round(max_hole_length,3),
                               run_time=np.round(run_time,4)
                              )



            dicts_to_insert.append(insert_dict)
        self.insert(dicts_to_insert,skip_duplicates=True)

# Running the Populate

In [15]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
(curr_table)#.delete()#.delete()# & "status='error'"#.delete()
#curr_table.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__baylor_segment_centroid,4cf132d0167641826772e58b9a86f35c,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,287786,321702,2021-04-14 13:02:01


In [16]:
import time
import random

sm = reload(sm)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=True)
else:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for BaylorSegmentCentroid populate = {time.time() - start_time}")

INFO - 2021-04-14 18:02:42,158 - autopopulate - Found 15 keys to populate
INFO - 2021-04-14 18:02:42,169 - connection - Transaction started
INFO - 2021-04-14 18:02:42,171 - autopopulate - Populating: {'segment_id': 864691135385570133, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691135385570133 ----
{'segment_id': 864691135385570133, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 75000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
max_mesh_sized_filtered_away = 22500.0
xvfb-run -n 949 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_61932.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_61932_remove_interior.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/remove_interior_414278.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_61932.off
re

/meshAfterParty/meshAfterParty/trimesh_utils.py:709: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


There were 19 total interior meshes
Pieces satisfying glia requirements (volume) (x >= 2500000000000): 0
Pieces satisfying nuclie requirements: n_faces (700 <= x) and volume (x < 2500000000000) : 19
inside remove_mesh_interior and using precomputed inside_pieces
Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(23175, 3), faces.shape=(49719, 3))>, <trimesh.Trimesh(vertices.shape=(10877, 3), faces.shape=(23680, 3))>, <trimesh.Trimesh(vertices.shape=(3636, 3), faces.shape=(8666, 3))>, <trimesh.Trimesh(vertices.shape=(2238, 3), faces.shape=(5105, 3))>, <trimesh.Trimesh(vertices.shape=(1780, 3), faces.shape=(4051, 3))>, <trimesh.Trimesh(vertices.shape=(1018, 3), faces.shape=(2306, 3))>, <trimesh.Trimesh(vertices.shape=(882, 3), faces.shape=(2021, 3))>, <trimesh.Trimesh(vertices.shape=(794, 3), faces.shape=(1764, 3))>, <trimesh.Trimesh(vertices.shape=(731, 3), faces.shape=(1637, 3))>, <trimesh.Trimesh(vertices.shape=(692, 3), faces.shape=(1540, 3))>, <trimesh.Trimesh(

/meshAfterParty/meshAfterParty/trimesh_utils.py:3012: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


...

Largest hole before segmentation = 75284.74849571647, after = 426307.37923317176,

ratio = 5.662599500580488, difference = 351022.63073745527
Saved object at /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137197169985_glia.pbz2
File size is 4.6e-05 MB
Saved object at /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137197169985_nuclei.pbz2
File size is 0.145278 MB
 glia_path = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137197169985_glia.pbz2 
 nuclei_path = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137197169985_nuclei.pbz2
Finished saving off glia and nuclei information : {'segment_id': 864691137197169985, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': 30, 'n_glia_faces': 0, 'glia_faces': PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137197169985_glia.pbz2'), 'n_nuclei_faces': 115571, 'nuclei_faces': PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/86469113719716998

/meshAfterParty/meshAfterParty/networkx_utils.py:593: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles_list_array = np.array(cycles_list)


Trying to write off file
Predicted Coordinates are [194865 121605  17038]


INFO - 2021-04-14 19:07:17,140 - connection - Transaction committed and closed.


Populate Done
Total time for BaylorSegmentCentroid populate = 3875.001152276993
